In [1]:
from datasets import load_dataset, Audio
import io
import soundfile as sf
import numpy as np

from datasets import load_dataset, Features, Value, Array2D, Array3D
from torch.utils.data import DataLoader as TorchLoader
import io
import soundfile as sf
import torch

import torch.nn.functional as F
from transformers import WhisperProcessor, WhisperForConditionalGeneration

So the main issue was that the hf decoder was not working because there was some issue with torchcodec. By disabling automatic decoding and manually reading the raw bytes, we can use soundfile or librosa to decode the audio data correctly.

Preprocessing classes -> HFDataweapper and ASRPipeline
fn -> collate_batch

In [2]:
class HFDataWrapper:
    def __init__(self, ds_split):
        self.ds = ds_split

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        sample = self.ds[idx]
        audio_info = sample["audio"]

        # manually decode bytes using soundfile
        raw_bytes = audio_info["bytes"]
        with io.BytesIO(raw_bytes) as f:
            waveform, sr = sf.read(f, dtype="float32")

        # convert to torch tensor
        if waveform.ndim == 1:
            waveform = torch.tensor(waveform).unsqueeze(0)
        else:
            waveform = torch.tensor(waveform.T)

        meta = {k: v for k, v in sample.items() if k != "audio"}
        return waveform, sr, meta


def collate_batch(batch):
    """Pads variable-length audio for batching"""
    waveforms, srs, metas = zip(*batch)
    srs = torch.tensor(srs)
    max_len = max(w.shape[1] for w in waveforms)
    padded = [torch.nn.functional.pad(w, (0, max_len - w.shape[1])) for w in waveforms]
    waveforms = torch.stack(padded)
    return waveforms, srs, metas

In [3]:
# Load dataset
ds = load_dataset("ARTPARK-IISc/vaani", "Goa_NorthSouthGoa", split="train")

# Disable auto-decoding to get raw bytes
ds = ds.cast_column("audio", Features({"path": Value("string"), "bytes": Value("binary")}))

# Wrap dataset
wrapped_ds = HFDataWrapper(ds)

# Create DataLoader
loader = TorchLoader(wrapped_ds, batch_size=4, shuffle=True, collate_fn=collate_batch)

print("✅ DataLoader ready")

Using the latest cached version of the dataset since ARTPARK-IISc/vaani couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'Goa_NorthSouthGoa' at C:\Users\aadit\.cache\huggingface\datasets\ARTPARK-IISc___vaani\Goa_NorthSouthGoa\0.0.0\dcb8ea6dd7050ec2ffb5e07542a33da7bef2a8fc (last modified on Tue Nov  4 02:35:59 2025).


Loading dataset shards:   0%|          | 0/44 [00:00<?, ?it/s]

✅ DataLoader ready


In [7]:
class ASRPipeline:
    def __init__(self, processor, model, device=None):
        self.processor = processor
        self.model = model
        self.device = device or "cpu"
        self.model.to(self.device)

    def transcribe_batch(self, waveforms, sample_rates):
        # waveforms shape: (batch, 1, time)
        if isinstance(waveforms, torch.Tensor):
            if waveforms.ndim == 3:
                waveforms = waveforms.squeeze(1)
            elif waveforms.ndim == 2:
                pass
            elif waveforms.ndim == 1:
                waveforms = waveforms.unsqueeze(0)
            else:
                raise ValueError(f"Unexpected waveform shape: {waveforms.shape}")
        else:
            raise ValueError("waveforms must be a torch.Tensor")

        # Convert each waveform to numpy
        waveforms = [w.cpu().numpy() for w in waveforms]
        sr = int(sample_rates[0]) if isinstance(sample_rates, (list, tuple, torch.Tensor)) else int(sample_rates)

        # Extract mel spectrograms (Whisper feature extractor)
        inputs = self.processor.feature_extractor(
            waveforms,
            sampling_rate=sr,
            return_tensors="pt",
            padding="longest"
        )

        input_features = inputs.input_features  # shape: (batch, 80, frames)

        # ✅ Fix: pad/truncate mel features to exactly 3000 frames
        B, n_mels, frames = input_features.shape
        target_len = 3000
        if frames < target_len:
            pad_amt = target_len - frames
            input_features = torch.nn.functional.pad(input_features, (0, pad_amt))
        elif frames > target_len:
            input_features = input_features[..., :target_len]

        input_features = input_features.to(self.device)

        # Inference
        with torch.no_grad():
            predicted_ids = self.model.generate(input_features=input_features)

        # Decode predictions
        transcriptions = self.processor.tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)
        return transcriptions


In [ ]:
model_name = "openai/whisper-small"

processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained(model_name)

asr = ASRPipeline(processor=processor, model=model, device="cpu")
print("✅ ASR Pipeline loaded on CPU")

✅ ASR Pipeline loaded on CPU


Simpler ASR pipeline class

In [ ]:
# Load the dataset
ds = load_dataset("ARTPARK-IISc/vaani", "Goa_NorthSouthGoa", split="train")

# Disable auto-decoding (keep raw audio bytes)
ds = ds.cast_column("audio", Features({"path": Value("string"), "bytes": Value("binary")}))

# Wrap dataset for PyTorch
wrapped_ds = HFDataWrapper(ds)


Using the latest cached version of the dataset since ARTPARK-IISc/vaani couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'Goa_NorthSouthGoa' at C:\Users\aadit\.cache\huggingface\datasets\ARTPARK-IISc___vaani\Goa_NorthSouthGoa\0.0.0\dcb8ea6dd7050ec2ffb5e07542a33da7bef2a8fc (last modified on Tue Nov  4 02:35:59 2025).


Loading dataset shards:   0%|          | 0/44 [00:00<?, ?it/s]

✅ DataLoader ready — batches will yield (waveforms, srs, metas)


In [14]:
# Build DataLoader
loader = TorchLoader(wrapped_ds, batch_size=4, shuffle=False, collate_fn=collate_batch)

print("✅ DataLoader ready — batches will yield (waveforms, srs, metas)")

✅ DataLoader ready — batches will yield (waveforms, srs, metas)


In [15]:
for waveforms, srs, metas in loader:
    transcriptions = asr.transcribe_batch(waveforms, srs)
    for meta, text in zip(metas, transcriptions):
        print(f"[{meta['language']}] {text}")
    break  # just first batch

[Konkani]  ಸಾರಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿ
[Konkani]  ಸಾರಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಿಲಿಲಿಿಲಿಿಲಿಿ
[Konkani]  तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो
[Konkani]  मागर इकुबमें साहा जिमान पुरर दिफरन ताएप्स


In [16]:
from IPython.display import Audio, display, HTML

# Limit to a few batches so it doesn’t spam your notebook
for batch_idx, (waveforms, srs, metas) in enumerate(loader):
    transcriptions = asr.transcribe_batch(waveforms, srs)

    print(f"\n🎯 Batch {batch_idx + 1}\n{'-' * 60}")
    for i, (meta, text, waveform) in enumerate(zip(metas, transcriptions, waveforms)):
        print(f"\n[{i + 1}] Language: {meta['language']}")
        print(f"Gender: {meta.get('gender', 'Unknown')}")
        print(f"District: {meta.get('district', 'N/A')}")
        print(f"Transcription: {text}")

        # Convert waveform to 1D NumPy array for playback
        audio_np = waveform.squeeze().cpu().numpy()
        display(Audio(audio_np, rate=int(srs[0])))

    # stop after 2 batches (remove to process full dataset)
    if batch_idx == 1:
        break


🎯 Batch 1
------------------------------------------------------------

[1] Language: Konkani
Gender: Male
District: NorthSouthGoa
Transcription:  ಸಾರಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿ



[2] Language: Konkani
Gender: Male
District: NorthSouthGoa
Transcription:  ಸಾರಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಿಲಿಲಿಿಲಿಿಲಿಿ



[3] Language: Konkani
Gender: Male
District: NorthSouthGoa
Transcription:  तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो तो



[4] Language: Konkani
Gender: Male
District: NorthSouthGoa
Transcription:  मागर इकुबमें साहा जिमान पुरर दिफरन ताएप्स



🎯 Batch 2
------------------------------------------------------------

[1] Language: Konkani
Gender: Male
District: NorthSouthGoa
Transcription:  අපි කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින් කින



[2] Language: Konkani
Gender: Male
District: NorthSouthGoa
Transcription:  my wearable lighting arrangements bare hathiton plus AC this thai AC asabiton and a carpet asa



[3] Language: Konkani
Gender: Male
District: NorthSouthGoa
Transcription:  अई ब्रो सुर्यास्त ता ज़ल्लो आसा



[4] Language: Konkani
Gender: Male
District: NorthSouthGoa
Transcription:  अना सर एक दिज्टिस्ता


In [17]:
from collections import Counter

# count languages in your wrapped dataset
lang_counts = Counter()
for i in range(len(wrapped_ds)):
    _, _, meta = wrapped_ds[i]
    lang = meta.get('language', 'Unknown')
    lang_counts[lang] += 1

print("📊 Language counts:")
for lang, count in lang_counts.items():
    print(f"{lang:10s} → {count}")

📊 Language counts:
Konkani    → 38530
Marathi    → 13194
Hindi      → 53761
Gujarati   → 94
English    → 49
Bengali    → 278
Kannada    → 119
